<a href="https://colab.research.google.com/github/sathyanarayanajammala/GenAI/blob/main/Langchain_Models_Prompts_Parsers_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative AI and Prompt Engineering
## A Program by IISc and TalentSprint



## **LangChain 🦜🔗: Models, Prompts, Output Parsers and Memory**

### Objectives:

At the end of the experiment you will be able to understand & use :
 1. Direct API call to OpenAI
 2. API calls through LangChain:
   * Prompts
   * Models
   * Output parsers
   * Memory

In [ ]:
!pip install openai
!pip install langchain

In [ ]:
import openai
import os

In [ ]:
f = open('/content/openapi_key.txt')
api_key = f.read().strip()          # Remove Blank Spaces
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key= os.getenv('OPENAI_API_KEY')

Note: LLMs do not always produce the same results. When executing the code in your notebook, you may get slightly different answers than the demo.

### **Chat API : OpenAI**

Let's start with a direct API call to OpenAI.

In [ ]:
client = openai.OpenAI()

def llm_response(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [ ]:
llm_response("What is 1+1?")

In [ ]:
customer_email = """
My washing machine stops after 15 minutes. \
Something is clogged in outlet or inlet water pipe. \
I need your help \
right now, buddy!
"""

In [ ]:
style = """Hindi \
in a calm and respectful tone
"""

In [ ]:
prompt = f"""Translate the text \
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

In [ ]:
response = llm_response(prompt)
response

### **Chat API : LangChain**

Let's try how we can do the same using LangChain.

#### **Model**

In [ ]:
!pip install langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI
# This is langchain abstraction for the chatGPT API endpoint

In [ ]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature = 0.0) # model=llm_model
chat

#### **Prompt template**

In [ ]:
template_s = """Translate the text \
into {style1}.\
text: ```{text1}```
"""

we can now repeadedly use this template:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
#from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_s)

In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
customer_style = """Hindi \
in a calm and respectful tone
"""

In [ ]:
customer_email = """
My washing machine stops after 15 minutes. \
Something is clogged in outlet or inlet water pipe. \
I need your help \
right now, buddy!
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    style1=customer_style,
                    text1=customer_email)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

In [ ]:
print(customer_messages[0])

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [ ]:
print(customer_response.content)

In [ ]:
service_reply = """इनलेट और आउटलेट नली खोलें\
और पाइप साफ करें\
सामने दाहिनी ओर नीचे एक नोब भी खोलें\
उसे भी साफ़ करो. \
आगे की कठिनाई के लिए हमसे संपर्क करें
"""

In [ ]:
service_style = """English \
in a calm and respectful tone
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style1=service_style,
    text1=service_reply)

print(service_messages[0].content)

In [ ]:
service_response = chat.invoke(service_messages)
print(service_response.content)

### **Output Parsers**

Given below is an example of customer review:

In [ ]:
customer_review_1 = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

From above customer review: we want to get information as given below:

In [ ]:
{
  "gift": True,
  "delivery_days": 2,
  "price_value": "slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features"
}

In [ ]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift or present for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
customer_review_2 = """\
This vacuum cleaner is absolutely fantastic. It comes with four different modes: light suction, medium breeze,\
 strong wind, and cyclone power. It was delivered within three days, just in time for my husband's birthday surprise.\
  I believe he was really impressed by it. Up to now, I've been the only one operating it, and I've been using it\
   regularly to tidy up the floors in our home. It does cost a bit more than other vacuum cleaners on the market,\
    but in my opinion, it's totally worth the investment considering its extra functionalities.
    """

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

In [ ]:
messages = prompt_template.format_messages(text=customer_review_1)
chat = ChatOpenAI(temperature=0.0)
response = chat.invoke(messages)
print(response.content)

In [ ]:
type(response.content)

In [ ]:
# You will get an error by running this line of code
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

#### **Parse the LLM output string into a Python dictionary**::
[Structured output parser](https://python.langchain.com/docs/modules/model_io/output_parsers/types/structured)

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

In [ ]:
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

In [ ]:
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

In [ ]:
response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

In [ ]:
prompt = ChatPromptTemplate.from_template(template=review_template)

messages = prompt.format_messages(text=customer_review_1,
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

In [ ]:
response = chat.invoke(messages)

In [ ]:
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)
output_dict

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('delivery_days')



### **LangChain: Memory**

LangChain can help in building better chatbots, or have
an LLM with more effective chats by better managing
what it remembers from the conversation you've had so far.

* [ConversationBufferMemory](https://python.langchain.com/docs/modules/memory/types/buffer)
* [ConversationBufferWindowMemory](https://python.langchain.com/docs/modules/memory/types/buffer_window):  It only uses the last K interactions.
* [ConversationTokenBufferMemory](https://python.langchain.com/docs/modules/memory/types/token_buffer): It uses token length rather than number of interactions to determine when to flush interactions.
* [ConversationSummaryMemory](https://python.langchain.com/docs/modules/memory/types/summary) : This type of memory creates a summary of the conversation over time.


#### **ConversationBufferMemory**
##### **[Customizing Conversational Memory](https://python.langchain.com/docs/modules/memory/conversational_customization)**

The **ConversationChain** is a class from the LangChain library that facilitates having a conversation and loading context from memory.

The primary purpose of the ConversationChain is to enable conversational interactions with a language model (LLM) while maintaining context across multiple turns.

It’s particularly useful for chatbots, virtual assistants, or any application where maintaining conversation history matters.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
llm_model="gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True # False
)

In [ ]:
conversation.predict(input="Hi,I am Ramendra")

In [ ]:
conversation.predict(input="What is 6 divided by 2?")

In [ ]:
conversation.predict(input="What is my name?")

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

How langchain keeps adding the conversation in memory?

In [ ]:
memory = ConversationBufferMemory()

In [ ]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

In [ ]:
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [ ]:
memory.load_memory_variables({})

#### **ConversationBufferWindowMemory**

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory = ConversationBufferWindowMemory(k=1)

In [ ]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [ ]:
memory.load_memory_variables({})

In [ ]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False # True
)

In [ ]:
conversation.predict(input="Hi,I am Ramendra")

In [ ]:
conversation.predict(input="What is 1+1?")

In [ ]:
conversation.predict(input="What is my name?")

#### **ConversationTokenBufferMemory**

In [ ]:
!pip install tiktoken

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationTokenBufferMemory
llm_model="gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [ ]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=20)# Different llm has different ways of counting tokens
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},
                    {"output": "Charming!"})

In [ ]:
memory.load_memory_variables({})

#### **ConversationSummaryMemory**

In the conversation summary buffer memory, instead of limiting the memory to a fixed number
of tokens based on the most recent utterances or a fixed number of conversational exchanges, it uses an LLM to write a summary of the conversation so far,
and let that be the memory.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"},
                    {"output": f"{schedule}"})

In [ ]:
memory.load_memory_variables({})

In [ ]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="What would be a good demo to show?")

In [ ]:
memory.load_memory_variables({})